In [1]:
from PIL import Image

读取Json文件，将坐标转换为yolo格式，然后将图片和标签分别保存到train.txt和test.txt中

In [2]:
import os
import json
root_data_path='data/train/'

In [12]:
import shutil
def locate_transfer(file,label):
    #将给出的绝对像素左上角坐标转换为yolo的中心点相对坐标
    img=Image.open(file)
    img_shape=img.size
    x_center=label[0]+label[2]//2
    y_center=label[1]+label[3]//2
    x_center=x_center/img_shape[0]
    y_center=y_center/img_shape[1]
    width=label[2]/img_shape[0]
    height=label[3]/img_shape[1]
    return f'0 {x_center} {y_center} {width} {height}'
    
video_folder_list=os.listdir(root_data_path)
for video_folder in video_folder_list:
    video_folder_path=os.path.join(root_data_path,video_folder)
    labels=json.load(open(os.path.join(video_folder_path,'IR_label.json')))
    #print(labels)
    exist=labels['exist']
    gt_rect=labels['gt_rect']
    for i in range(len(exist)):
        if exist[i]==1:# 目标存在
            filename=str(i+1).zfill(6)
            # 拷贝图片
            shutil.copy(os.path.join(video_folder_path,filename+'.jpg'),os.path.join('dataset','images','train',video_folder+'_'+filename+'.jpg'))
            # 生成标注
            label=gt_rect[i]
            label=locate_transfer(os.path.join(video_folder_path,filename+'.jpg'),label)
            with open(os.path.join('dataset','labels','train',video_folder+'_'+filename+'.txt'),'w+') as f:
                f.write(label)
    

In [5]:
# 将部分train数据集转移到val数据集
from random import shuffle
import shutil
import os

val_rate=0.1
train_img_path='dataset/images/train/'
train_label_path='dataset/labels/train/'
val_img_path='dataset/images/val/'
val_label_path='dataset/labels/val/'

img_list=os.listdir(train_img_path)
shuffle(img_list) # 打乱顺序
val_num=int(len(img_list)*val_rate)
for i in range(val_num):
    img_name=img_list[i]
    shutil.move(os.path.join(train_img_path,img_name),os.path.join(val_img_path,img_name))
    shutil.move(os.path.join(train_label_path,img_name[:-4]+'.txt'),os.path.join(val_label_path,img_name[:-4]+'.txt'))